In [397]:
import random

class Sentiment:
    NEGETIVE = 'NEGETIVE'
    NEUTRAL = 'NEUTRAL'
    POSITIVE = 'POSITIVE'


class Review:
    def __init__(self,text,score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.score <=2:
            return Sentiment.NEGETIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else:
            return Sentiment.POSITIVE
class ReviewContainer:
    def __init__(self,reviews):
        self.reviews = reviews
    
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
        
    def evenly_distributed(self):
        negetive = list(filter(lambda x:x.sentiment == Sentiment.NEGETIVE , self.reviews))
        positive = list(filter(lambda x:x.sentiment == Sentiment.POSITIVE , self.reviews))
        
        positive_shrink = positive[:len(negetive)]
        self.reviews = negetive + positive_shrink
        random.shuffle(self.reviews)
        
        

In [398]:
import json

file_name = 'Books_small_10000.json'

reviews = []

with open(file_name) as f:
    for line in f:
        #print(line)
        review = json.loads(line)
        
        
        #print(review['reviewText'])
        #print(review['overall'])
        reviews.append(Review(review['reviewText'],review['overall']))

In [399]:
reviews[5].score

5.0

In [400]:
reviews[5].text

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

In [401]:
reviews[5].sentiment

'POSITIVE'

In [402]:
len(reviews)

10000

In [403]:
from sklearn.model_selection import train_test_split

In [404]:
training,test = train_test_split(reviews,test_size = 0.33, random_state = 42 )

train_container = ReviewContainer(training)

test_container = ReviewContainer(test)

cont.evenly_distributed()
len(cont.reviews)

872

In [405]:
len(training)

6700

In [406]:
len(test)

3300

In [407]:
len(test)

3300

In [408]:
print(training[0].sentiment)

POSITIVE


In [409]:
train_container.evenly_distributed()
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

train_container.evenly_distributed()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

print(train_y.count(Sentiment.POSITIVE))
print(train_y.count(Sentiment.NEGETIVE))



436
436


In [410]:
train_x[0]

'A good mystery! A chance meeting beings two people back hurtful memories from twenty some years ago.Lies and threat bring confusion and mistrust !'

In [411]:
train_y[0]

'POSITIVE'

In [412]:
test_x[0]

"was sent an Arc of this book for an honest review and here it is = This is the kind of book that you want to read while sitting in front of the fire with a cup of hot apple cider and a blanket over your legs.I have read many of Jaci Burton's books and have never been disappointed. This first book in her new Hope series does not disappoint either.This is the story of Emma, a new vet who has come back home to open her own practice and Luke McCormack, a police officer in the same town.Both have been previously burned by love so both have issues but, that doesn't stop them from acting on that attraction.This book pulls you in from the first page, wraps you up and doesn't let you go until the end.I loved it!"

In [413]:
test_y[0]

'POSITIVE'

### Bag of word vectorization

In [454]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

# This book is great
# This book was so bad

    
vectorizer = TfidfVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)
test_x_vectors = vectorizer.transform(test_x)


    

In [455]:
train_x_vectors.shape


(872, 8906)

### Classification
linear SVM

In [456]:
from sklearn import svm

clf_svm = svm.SVC(kernel='linear')
clf_svm = clf_svm.fit(train_x_vectors, train_y)


In [457]:
test_x[0]

"was sent an Arc of this book for an honest review and here it is = This is the kind of book that you want to read while sitting in front of the fire with a cup of hot apple cider and a blanket over your legs.I have read many of Jaci Burton's books and have never been disappointed. This first book in her new Hope series does not disappoint either.This is the story of Emma, a new vet who has come back home to open her own practice and Luke McCormack, a police officer in the same town.Both have been previously burned by love so both have issues but, that doesn't stop them from acting on that attraction.This book pulls you in from the first page, wraps you up and doesn't let you go until the end.I loved it!"

In [458]:
clf_svm.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

### Decision Tree

In [459]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec = clf_dec.fit(train_x_vectors, train_y)
clf_dec.predict(test_x_vectors[0])



array(['NEGETIVE'], dtype='<U8')

### Neive Bayes

In [460]:
from sklearn.naive_bayes import GaussianNB
clf_gnb = DecisionTreeClassifier()
clf_gnb = clf_dec.fit(train_x_vectors, train_y)
clf_gnb.predict(test_x_vectors[0])


array(['NEGETIVE'], dtype='<U8')

### Logistic Regression

In [461]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()
clf_log = clf_dec.fit(train_x_vectors, train_y)
clf_log.predict(test_x_vectors[0])

array(['NEGETIVE'], dtype='<U8')

## Evaluation
##### mean accuracy

In [462]:
clf_svm.score(test_x_vectors,test_y)

0.7412121212121212

In [463]:
clf_dec.score(test_x_vectors,test_y)

0.5624242424242424

In [464]:
clf_gnb.score(test_x_vectors,test_y)

0.5624242424242424

In [465]:
clf_log.score(test_x_vectors,test_y)

0.5624242424242424

### F1 score

In [466]:
from sklearn.metrics import f1_score

f1_score(test_y,clf_svm.predict(test_x_vectors),average=None,labels=[Sentiment.POSITIVE,Sentiment.NEGETIVE])

array([0.87656461, 0.3142329 ])

In [467]:
f1_score(test_y,clf_dec.predict(test_x_vectors),average=None,labels=[Sentiment.POSITIVE,Sentiment.NEUTRAL,Sentiment.NEGETIVE])

C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


array([0.73319149, 0.        , 0.16888889])

In [468]:
f1_score(test_y,clf_gnb.predict(test_x_vectors),average=None,labels=[Sentiment.POSITIVE,Sentiment.NEUTRAL,Sentiment.NEGETIVE])

array([0.73319149, 0.        , 0.16888889])

In [469]:
f1_score(test_y,clf_log.predict(test_x_vectors),average=None,labels=[Sentiment.POSITIVE,Sentiment.NEUTRAL,Sentiment.NEGETIVE])

array([0.73319149, 0.        , 0.16888889])

In [430]:
train_y.count(Sentiment.POSITIVE)

436

In [431]:
train_y.count(Sentiment.NEUTRAL)

0

In [432]:
train_y.count(Sentiment.NEGETIVE)

436

In [453]:
test_set = ['quality is  not good', 'awesome book','good','not great' , 'not too bad', 'very good','very fun']
new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)

array(['NEGETIVE', 'POSITIVE', 'POSITIVE', 'NEGETIVE', 'NEGETIVE',
       'POSITIVE', 'POSITIVE'], dtype='<U8')

#### Tuinig Our Model (with Grid Search)

In [470]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel':('linear','rbf'),'C': (1,4,8,16,32)}
svc = svm.SVC()

clf = GridSearchCV(svc,parameters,cv=5)
clf.fit(train_x_vectors,train_y)

C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: 

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': (1, 4, 8, 16, 32), 'kernel': ('linear', 'rbf')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [472]:
print(clf.score(test_x_vectors,test_y))

0.7412121212121212


#### Saving Model

In [480]:
import pickle

with open('./models/sentiment_classifier.pkl','wb') as f:
    pickle.dump(clf,f)
    

In [481]:
# Load Model



with open('./models/sentiment_classifier.pkl','rb') as f:
    loaded_clf = pickle.load(f)

In [482]:
print(test_x[0])

loaded_clf.predict(test_x_vectors[0])

was sent an Arc of this book for an honest review and here it is = This is the kind of book that you want to read while sitting in front of the fire with a cup of hot apple cider and a blanket over your legs.I have read many of Jaci Burton's books and have never been disappointed. This first book in her new Hope series does not disappoint either.This is the story of Emma, a new vet who has come back home to open her own practice and Luke McCormack, a police officer in the same town.Both have been previously burned by love so both have issues but, that doesn't stop them from acting on that attraction.This book pulls you in from the first page, wraps you up and doesn't let you go until the end.I loved it!


array(['POSITIVE'], dtype='<U8')